# Setting Parameters

In [133]:
lsh_threshold = 0.2  # Adjust as needed

# Importing libs

In [134]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
import re
import time


# Preprocessing

In [135]:
# Load your data
df1 = pd.read_csv('../data/matchto-db-records.csv')
df2 = pd.read_csv('../data/source-companies.csv')

In [136]:
df1.head(2)

,id,name,address_1,address_2,city,state,zip_code,country,website_url
0,11699,Newburyport Five Cents Savings Bank Inc,63 State St,NaN,Newburyport,MA,01950-6615,USA,www.newburyportbank.com
1,77176,"Tensar International, LLC",2500 Northwinds Pkwy # 50,NaN,Alpharetta,GA,30009-2243,USA,www.tensarcorp.com


In [137]:
df1 = df1[['name', 'website_url']]

In [138]:
df1.isna().sum()

name              0
website_url    1887
dtype: int64

In [139]:
df1.columns = ['company_1', 'website_1']

In [140]:
df1.head(2)

,company_1,website_1
0,Newburyport Five Cents Savings Bank Inc,www.newburyportbank.com
1,"Tensar International, LLC",www.tensarcorp.com


In [141]:
df1.website_1.value_counts()

website_1
www.wellsfargo.com             21
www.va.gov                     13
www.chase.com                  11
www.jpmorgan.com               10
www.att.com                    10
                               ..
www.rojekconsulting.com         1
www.southernenvironment.org     1
www.hdtvpromotions.com          1
www.cdc.gov                     1
www.brantondejong.com           1
Name: count, Length: 7383, dtype: int64

In [142]:
df1.head(2)

,company_1,website_1
0,Newburyport Five Cents Savings Bank Inc,www.newburyportbank.com
1,"Tensar International, LLC",www.tensarcorp.com


### df1 preprocess complete

In [143]:
df2.head(2)

,company_name,website,address_1,city,state,zip
0,23andMe,23andme.com,899 W Evelyn Ave,Mountain View,CA,94041.0
1,280 Group,280group.com,142B S Santa Cruz Ave,Los Gatos,CA,95030.0


In [146]:
df2.columns = ['company_2', 'website_2', 'address_2', 'city_2', 'state_2', 'zip_code_2']
df2 = df2[['company_2', 'website_2']]

In [147]:
df2.head()

,company_2,website_2
0,23andMe,23andme.com
1,280 Group,280group.com
2,5AM Ventures,5amventures.com
3,Accel,www.accel.com
4,Addepar,addepar.com


for col in df1.columns:
    df1[col] = df1[col].apply(preprocess)

for col in df2.columns:
    df2[col] = df2[col].apply(preprocess)

In [148]:
df1.drop_duplicates(inplace=True)
df2.drop_duplicates(inplace=True)

In [149]:
df1.head(2)

,company_1,website_1
0,Newburyport Five Cents Savings Bank Inc,www.newburyportbank.com
1,"Tensar International, LLC",www.tensarcorp.com


In [150]:
df2.head(2)

,company_2,website_2
0,23andMe,23andme.com
1,280 Group,280group.com


In [151]:
for dataframe in [df1,df2]:
    for col in dataframe.columns:
        dataframe[col]=dataframe[col].apply(lambda x: str(x).lower())

In [152]:
df1.website_1 = df1.website_1.apply(lambda x: " ".join(x.split('.')))
df2.website_2 = df2.website_2.apply(lambda x: " ".join(x.split('.')))

In [153]:
df1['concat_1']=df1.apply(lambda row: ' '.join(map(str, row)), axis=1)
df2['concat_2']=df2.apply(lambda row: ' '.join(map(str, row)), axis=1)

# Create all possible combos

In [154]:
# Performing a cross join
df1['key'] = 1
df2['key'] = 1
df = pd.merge(df1, df2, on='key').drop('key', axis=1)

In [155]:
df.head(2)

,company_1,website_1,concat_1,company_2,website_2,concat_2
0,newburyport five cents savings bank inc,www newburyportbank com,newburyport five cents savings bank inc www ne...,23andme,23andme com,23andme 23andme com
1,newburyport five cents savings bank inc,www newburyportbank com,newburyport five cents savings bank inc www ne...,280 group,280group com,280 group 280group com


In [156]:
df1.head(2)

,company_1,website_1,concat_1,key
0,newburyport five cents savings bank inc,www newburyportbank com,newburyport five cents savings bank inc www ne...,1
1,"tensar international, llc",www tensarcorp com,"tensar international, llc www tensarcorp com",1


In [157]:
df1.shape

(9702, 4)

In [158]:
df.concat_1.nunique()

9677

In [159]:
df2.head(2)

,company_2,website_2,concat_2,key
0,23andme,23andme com,23andme 23andme com,1
1,280 group,280group com,280 group 280group com,1


In [160]:
df2.shape

(262, 4)

In [161]:
df2.concat_2.nunique()

262

In [162]:
df.head(3)

,company_1,website_1,concat_1,company_2,website_2,concat_2
0,newburyport five cents savings bank inc,www newburyportbank com,newburyport five cents savings bank inc www ne...,23andme,23andme com,23andme 23andme com
1,newburyport five cents savings bank inc,www newburyportbank com,newburyport five cents savings bank inc www ne...,280 group,280group com,280 group 280group com
2,newburyport five cents savings bank inc,www newburyportbank com,newburyport five cents savings bank inc www ne...,5am ventures,5amventures com,5am ventures 5amventures com


In [163]:
df['website_1'] = df['website_1'].str.replace('com', '').str.replace('www', '')
df['website_2'] = df['website_2'].str.replace('com', '').str.replace('www', '')


In [164]:
df.head(2)

,company_1,website_1,concat_1,company_2,website_2,concat_2
0,newburyport five cents savings bank inc,newburyportbank,newburyport five cents savings bank inc www ne...,23andme,23andme,23andme 23andme com
1,newburyport five cents savings bank inc,newburyportbank,newburyport five cents savings bank inc www ne...,280 group,280group,280 group 280group com


# Computing MinHash

In [165]:
# Function to create MinHash from a company name
def create_minhash(company):
    company = str(company)
    tokens = re.sub(r'\W+', ' ', company.lower()).split()
    m = MinHash()
    for token in tokens:
        m.update(token.encode('utf8'))
    return m

# Compute MinHashes for unique companies in both dataframes
unique_companies_1 = df1['concat_1'].unique()
unique_companies_2 = df2['concat_2'].unique()

minhashes_1 = {company: create_minhash(company) for company in unique_companies_1}
minhashes_2 = {company: create_minhash(company) for company in unique_companies_2}

# Compare hashes for all possible combinations
matches = []

for company1, minhash1 in minhashes_1.items():
    for company2, minhash2 in minhashes_2.items():
        if minhash1.jaccard(minhash2) > lsh_threshold:
            matches.append((company1, company2))

# Output matches
print(len(matches))

319913


In [166]:
concat_df = pd.DataFrame(matches, columns=['concat_1', 'concat_2'])


In [167]:
result_1 = pd.merge(concat_df,df, on=['concat_1', 'concat_2'])

In [168]:
result_1.concat_1.nunique()

7716

In [169]:
result_1.concat_2.nunique()

256

# Fuzzy Match

In [170]:
from fuzzywuzzy import fuzz

In [171]:
def calculate_fuzzy_score(column1, column2):
    return fuzz.token_set_ratio(str(column1), str(column2))

# List of columns to compare
columns_to_compare = ['company', 'website']

# Create new columns for fuzzy match scores
for column in columns_to_compare:
    result_1[f'{column}_score'] = result_1.apply(lambda x: calculate_fuzzy_score(x[f'{column}_1'], x[f'{column}_2']), axis=1)


In [172]:
result_1.columns

Index(['concat_1', 'concat_2', 'company_1', 'website_1', 'company_2',
       'website_2', 'company_score', 'website_score'],
      dtype='object')

In [173]:
result_1.sort_values(by=['company_score', 'website_score'], ascending=False,inplace=True)

In [174]:
final_df = result_1[[
    'company_1',
    'company_2',
    'company_score',
    'website_1',
    'website_2',
    'website_score',
]]

In [175]:
final_df.head(10)

,company_1,company_2,company_score,website_1,website_2,website_score
48489,google llc,google,100,google,google,100
54980,asus computer international inc,asus,100,asus tw,asus,100
92080,paypal credit,paypal,100,paypal,paypal,100
96835,apple inc.,apple,100,apple,apple,100
166852,intuit inc.,intuit,100,intuit,intuit,100
199501,intuit inc.,intuit,100,https:// intuit /privacy/statement/,intuit,100
224419,intel corporation,intel,100,intel,intel,100
275223,"synopsys, inc.",synopsys,100,synopsys,synopsys,100
303821,"mongodb, inc.",mongodb,100,mongodb,mongodb,100
133249,"fanatics, llc",fanatics,100,footballfanatics,fanatics,67


In [176]:
final_df['fuzz_index'] = 0.3*final_df.company_score+0.7*final_df.website_score

/var/folders/qd/53v0q6g93r5g_dc_bwl8_tlw0000gn/T/ipykernel_85395/2254232016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['fuzz_index'] = 0.3*final_df.company_score+0.7*final_df.website_score


In [177]:
final_df.sort_values(by='fuzz_index',ascending=False,inplace=True)

/var/folders/qd/53v0q6g93r5g_dc_bwl8_tlw0000gn/T/ipykernel_85395/2014474883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.sort_values(by='fuzz_index',ascending=False,inplace=True)


In [178]:
final_df.columns

Index(['company_1', 'company_2', 'company_score', 'website_1', 'website_2',
       'website_score', 'fuzz_index'],
      dtype='object')

In [179]:
match_df = final_df.sort_values('fuzz_index', ascending=False).groupby(['company_2','website_2']).head(1).reset_index(drop=True)


In [180]:
match_df.sort_values(by='fuzz_index')

,company_1,company_2,company_score,website_1,website_2,website_score,fuzz_index
255,fivethirtyeight,sumo logic,8,fivethirtyeight,sumologic,8,8.0
254,schlage lock co,elastic,18,nan,elastic co,15,15.9
253,puccimanuli,hewlett packard,23,puccimanuli,hp,15,17.4
252,savvyinsider.com,andreessen horowitz,11,nan,a16z,29,23.6
251,horchow,invoice2go,35,horchow,2go,20,24.5
...,...,...,...,...,...,...,...
4,"synopsys, inc.",synopsys,100,synopsys,synopsys,100,100.0
3,intel corporation,intel,100,intel,intel,100,100.0
2,intuit inc.,intuit,100,intuit,intuit,100,100.0
1,apple inc.,apple,100,apple,apple,100,100.0


In [181]:
match_df.shape

(256, 7)

In [182]:
match_df.to_csv('text_matching_data.csv',index=False)

# annotated data

In [183]:
match_df.sort_values(by='fuzz_index',ascending=False,inplace=True)
match_df.head(20)

,company_1,company_2,company_score,website_1,website_2,website_score,fuzz_index
0,google llc,google,100,google,google,100,100.0
5,"mongodb, inc.",mongodb,100,mongodb,mongodb,100,100.0
7,paypal credit,paypal,100,paypal,paypal,100,100.0
6,asus computer international inc,asus,100,asus tw,asus,100,100.0
1,apple inc.,apple,100,apple,apple,100,100.0
4,"synopsys, inc.",synopsys,100,synopsys,synopsys,100,100.0
2,intuit inc.,intuit,100,intuit,intuit,100,100.0
3,intel corporation,intel,100,intel,intel,100,100.0
8,"mig capital management, inc.",makena capital management,84,nan,nan,100,95.2
9,jps partners inc,gi partners,84,nan,nan,100,95.2


In [54]:
true_data = pd.read_csv("../export_data (1).csv")

In [55]:
true_data.head(1)

,job_id,company_1,company_2,company_score,address_1,address_2,address_score,city_1,city_2,city_score,website_1,website_2,website_score,state_1,state_2,state_score,fuzz_index,Annotation
0,08c045ba-7ffb-48bd-ba20-dc2028c8da25,fremont,fremont,ca,ca,48720 kato rd,48720 kato rd,asus computer international inc,asus,asus tw,asus,100,100.0,100,100,100,100,"{""Text Matching"": {""match"": ""Match""}}"


In [67]:
# Check for missing values
if true_data['Annotation'].isnull().any():
    # Handle or remove missing values as needed
    true_data = true_data.dropna(subset=['Annotation'])

# Check the structure of dictionaries
def extract_text_matching(annotation):
    if isinstance(annotation, dict) and 'Text Matching' in annotation:
        # Check if 'Text Matching' key exists in the dictionary
        return annotation['Text Matching'].get('match', 'Not Found')
    else:
        return 'Not a dictionary or missing key'

# Apply the function to create a new 'match' column
true_data['match'] = true_data['Annotation'].apply(extract_text_matching)


In [69]:
true_data['label'] = (true_data.match=="Match").astype(int)

In [71]:
true_data.columns

Index(['job_id', 'company_1', 'company_2', 'company_score', 'address_1',
       'address_2', 'address_score', 'city_1', 'city_2', 'city_score',
       'website_1', 'website_2', 'website_score', 'state_1', 'state_2',
       'state_score', 'fuzz_index', 'Annotation', 'match', 'label'],
      dtype='object')

In [72]:
true_data.columns = ['job_id', 'city_1', 'city_2', 'state_1', 'state_2','address_1',
       'address_2','company_1', 'company_2', 'website_1', 'website_2', 'company_score',  'address_score', 'city_score',
       'website_score', 
       'state_score', 'fuzz_index', 'Annotation', 'match', 'label']

In [73]:
true_data.head(2)

,job_id,city_1,city_2,state_1,state_2,address_1,address_2,company_1,company_2,website_1,website_2,company_score,address_score,city_score,website_score,state_score,fuzz_index,Annotation,match,label
0,08c045ba-7ffb-48bd-ba20-dc2028c8da25,fremont,fremont,ca,ca,48720 kato rd,48720 kato rd,asus computer international inc,asus,asus tw,asus,100,100.0,100,100,100,100,{'Text Matching': {'match': 'Match'}},Match,1
1,9f0e06ff-e88a-4301-8fb0-6093c68aa0fd,santa clara,santa clara,ca,ca,2200 mission college blvd,2200 mission college blvd,intel corporation,intel,intel,intel,100,100.0,100,100,100,100,{'Text Matching': {'match': 'Match'}},Match,1


In [74]:
true_data = true_data[['city_1', 'city_2', 'state_1', 'state_2','address_1',
       'address_2','company_1', 'company_2', 'website_1', 'website_2','label']]

In [76]:
true_data.label.sum()

8

In [82]:
df2 = pd.merge(true_data,match_df,on = ['city_1', 'city_2', 'state_1', 'state_2','address_1',
       'address_2','company_1', 'company_2', 'website_1', 'website_2',],how = 'left')

In [83]:
df2.isna().sum()

city_1            1
city_2            3
state_1           3
state_2           4
address_1         4
address_2         5
company_1         0
company_2         0
website_1        43
website_2        37
label             0
company_score    50
address_score    50
city_score       50
website_score    50
state_score      50
fuzz_index       50
dtype: int64

In [84]:
true_data[true_data.label==1]

,city_1,city_2,state_1,state_2,address_1,address_2,company_1,company_2,website_1,website_2,label
0,fremont,fremont,ca,ca,48720 kato rd,48720 kato rd,asus computer international inc,asus,asus tw,asus,1
1,santa clara,santa clara,ca,ca,2200 mission college blvd,2200 mission college blvd,intel corporation,intel,intel,intel,1
2,mountain view,mountain view,ca,ca,690 e middlefield rd,690 e middlefield rd,"synopsys, inc.",synopsys,synopsys,synopsys,1
3,cupertino,cupertino,ca,ca,1 apple park way,apple campus,apple inc.,apple,apple,apple,1
4,mountain view,mountain view,ca,ca,2535 garcia ave,2675 coast ave,intuit inc.,intuit,intuit,intuit,1
13,san mateo,san mateo,ca,ca,1100 park pl ste 300,1100 park pl #300,alienvault llc,alienvault,cybersecurity att,alienvault,1
26,sunnyvale,sunnyvale,ca,ca,701 first ave,701 1st ave,"yahoo cv, llc",yahoo,verizonmedia,yahoo,1
73,colorado springs,san jose,co,NaN,2424 garden of the gods rd,6280 america center dr,perspecta enterprise solutions llc,hewlett-packard enterprise,hpe,hpe,1


# Total evaluation

In [90]:
df.shape

(2617118, 12)

In [122]:
state_city_match = df[(df.state_1==df.state_2) & (df.city_1==df.city_2)]

In [123]:
state_city_match

,company_1,address_1,city_1,state_1,website_1,concat_1,company_2,address_2,city_2,state_2,website_2,concat_2
25685,emerson equity llc,155 bovet rd ste 725,san mateo,ca,www emersonequity com,emerson equity llc 155 bovet rd ste 725 san ma...,alienvault,1100 park pl #300,san mateo,ca,www alienvault com,alienvault 1100 park pl #300 san mateo ca www ...
25695,emerson equity llc,155 bovet rd ste 725,san mateo,ca,www emersonequity com,emerson equity llc 155 bovet rd ste 725 san ma...,apttus,1400 fashion island blvd #100,san mateo,ca,apttus com,apttus 1400 fashion island blvd #100 san mateo...
25697,emerson equity llc,155 bovet rd ste 725,san mateo,ca,www emersonequity com,emerson equity llc 155 bovet rd ste 725 san ma...,armory,28 e 3rd ave #200,san mateo,ca,www armory io,armory 28 e 3rd ave #200 san mateo ca www armo...
25704,emerson equity llc,155 bovet rd ste 725,san mateo,ca,www emersonequity com,emerson equity llc 155 bovet rd ste 725 san ma...,benevity,32 w 25th ave #203,san mateo,ca,benevity com,benevity 32 w 25th ave #203 san mateo ca benev...
25743,emerson equity llc,155 bovet rd ste 725,san mateo,ca,www emersonequity com,emerson equity llc 155 bovet rd ste 725 san ma...,dokkio,1 waters park dr #103,san mateo,ca,dokkio com,dokkio 1 waters park dr #103 san mateo ca dokk...
...,...,...,...,...,...,...,...,...,...,...,...,...
2617048,dejong branton & associates,6155 almaden expy ste 350,san jose,ca,www brantondejong com,dejong branton & associates 6155 almaden expy ...,securly,111 n market st 4th floor,san jose,ca,securly com,securly 111 n market st 4th floor san jose ca ...
2617077,dejong branton & associates,6155 almaden expy ste 350,san jose,ca,www brantondejong com,dejong branton & associates 6155 almaden expy ...,taos,121 daggett dr,san jose,ca,taos com,taos 121 daggett dr san jose ca taos com
2617080,dejong branton & associates,6155 almaden expy ste 350,san jose,ca,www brantondejong com,dejong branton & associates 6155 almaden expy ...,tenable,nan,san jose,ca,www tenable com,tenable nan san jose ca www tenable com
2617106,dejong branton & associates,6155 almaden expy ste 350,san jose,ca,www brantondejong com,dejong branton & associates 6155 almaden expy ...,xilinx,2050 logic dr,san jose,ca,xilinx com,xilinx 2050 logic dr san jose ca xilinx com


In [128]:
def calculate_fuzzy_score(column1, column2):
    return fuzz.partial_ratio(str(column1), str(column2))

# List of columns to compare
columns_to_compare = ['company', 'address','website']

# Create new columns for fuzzy match scores
for column in columns_to_compare:
    state_city_match[f'{column}_score'] = state_city_match.apply(lambda x: calculate_fuzzy_score(x[f'{column}_1'], x[f'{column}_2']), axis=1)


/var/folders/qd/53v0q6g93r5g_dc_bwl8_tlw0000gn/T/ipykernel_85395/4105871683.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_city_match[f'{column}_score'] = state_city_match.apply(lambda x: calculate_fuzzy_score(x[f'{column}_1'], x[f'{column}_2']), axis=1)


In [131]:
state_city_match.sort_values(by = ['website_score','company_score','address_score',],ascending = False,inplace=True)

/var/folders/qd/53v0q6g93r5g_dc_bwl8_tlw0000gn/T/ipykernel_85395/1859811242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_city_match.sort_values(by = ['website_score','company_score','address_score',],ascending = False,inplace=True)


In [132]:
state_city_match.head(20)

,company_1,address_1,city_1,state_1,website_1,concat_1,company_2,address_2,city_2,state_2,website_2,concat_2,company_score,address_score,website_score
428917,asus computer international inc,48720 kato rd,fremont,ca,www asus com tw,asus computer international inc 48720 kato rd ...,asus,48720 kato rd,fremont,ca,www asus com,asus 48720 kato rd fremont ca www asus com,100,100,100
1801366,intel corporation,2200 mission college blvd,santa clara,ca,www intel com,intel corporation 2200 mission college blvd sa...,intel,2200 mission college blvd,santa clara,ca,intel com,intel 2200 mission college blvd santa clara ca...,100,100,100
2221717,"synopsys, inc.",690 e middlefield rd,mountain view,ca,www synopsys com,"synopsys, inc. 690 e middlefield rd mountain v...",synopsys,690 e middlefield rd,mountain view,ca,synopsys com,synopsys 690 e middlefield rd mountain view ca...,100,100,100
1327149,intuit inc.,2535 garcia ave,mountain view,ca,www intuit com,intuit inc. 2535 garcia ave mountain view ca w...,intuit,2675 coast ave,mountain view,ca,www intuit com,intuit 2675 coast ave mountain view ca www int...,100,62,100
894748,apple inc.,1 apple park way,cupertino,ca,www apple com,apple inc. 1 apple park way cupertino ca www a...,apple,apple campus,cupertino,ca,www apple com,apple apple campus cupertino ca www apple com,100,58,100
1077692,portola partners,1550 el camino real ste 20,menlo park,ca,nan,portola partners 1550 el camino real ste 20 me...,gi partners,2180 sand hill road,menlo park,ca,nan,gi partners 2180 sand hill road menlo park ca nan,82,37,100
1077646,portola partners,1550 el camino real ste 20,menlo park,ca,nan,portola partners 1550 el camino real ste 20 me...,canaan partners,2765 sand hill road,menlo park,ca,nan,canaan partners 2765 sand hill road menlo park...,77,32,100
1077711,portola partners,1550 el camino real ste 20,menlo park,ca,nan,portola partners 1550 el camino real ste 20 me...,highland capital partners,2420 sand hill road suite 300,menlo park,ca,nan,highland capital partners 2420 sand hill road ...,75,43,100
1077809,portola partners,1550 el camino real ste 20,menlo park,ca,nan,portola partners 1550 el camino real ste 20 me...,silver lake partners,2775 sand hill road,menlo park,ca,nan,silver lake partners 2775 sand hill road menlo...,75,32,100
1077724,portola partners,1550 el camino real ste 20,menlo park,ca,nan,portola partners 1550 el camino real ste 20 me...,interwest partners,2710 sand hill road,menlo park,ca,nan,interwest partners 2710 sand hill road menlo p...,69,37,100
